In [1]:
import pandas as pd
import ca_util as ca
import datetime as dt
import sys
import csv
import pickle
import tensorflow as tf
import inspect_checkpoint
from sklearn import preprocessing


config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session=tf.Session(config=config)


In [2]:

# 데이터 전처리 환경 저장한 파일을 불러 옵니다.
with open('./output/preprocess_object.pickle', 'rb') as f:
	preprocess_object = pickle.load(f)

# 훈련데이터 로드 합니다.
dataset = pd.read_csv('./output/test_dataset.csv', delimiter=',' ,encoding='utf-8', low_memory=False)



In [3]:

# 연속형 항목 전처리(표준화) 정보 불러와 적용합니다. 
number_df = dataset.select_dtypes(include=['int64', 'float64'])
scaler = preprocess_object.pop('scaler',None)

# 표준화 적용 & DataFrade/Column 명 적용합니다.
scaler_arr = scaler.transform(number_df)
org_data = pd.DataFrame(scaler_arr, columns=number_df.columns)

# 이산형 데이터 OneHot Encoder 처리 합니다.
categorical = list(dataset.select_dtypes(include=['object']).columns)
oh_encoder = {}

for cat in categorical:
	# OneHot 정보 불러옵니다.
	oh_encoder = preprocess_object[cat]
	# LabelEncoder & OneHot Encoder 불러 옵니다.
	le = oh_encoder['LabelEncoder']
	ohe = oh_encoder['OneHotEncoder']
	# 범주형 LabelEncoder 처리 합니다.
	cat_le_arr = le.transform(dataset[cat])
	cat_ohe_arr = ohe.transform(cat_le_arr.reshape(-1,1)).toarray()
	oh_df = pd.DataFrame(cat_ohe_arr, columns=[cat+'_'+le.inverse_transform([int(i)])[0] for i in range(cat_ohe_arr.shape[1])])
	# OneHot 결과 데이터 추가 합니다.
	org_data = pd.concat([org_data, oh_df], axis=1)

# 반응결과(종속변수) 새로운 데이터셋 분리 합니다.
test_label = pd.DataFrame()
test_label['resp_N'] = org_data['resp_N']
test_label['resp_Y'] = org_data['resp_Y']

# 데이터셋(독립변수) 반응결과 삭제 합니다.
del org_data['resp_N']
del org_data['resp_Y']


In [4]:

# index 재정렬 후 삭제 합니다.(drop 하지 않아 현재는 필요 없습니다.)
# org_data = org_data.reset_index()
# del org_data['index']

# DNN 모델 설정 합니다.
X = tf.placeholder(tf.float32, [None, org_data.shape[1]])
Y = tf.placeholder(tf.float32, [None, test_label.shape[1]])
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([org_data.shape[1], 256], stddev=0.01))
L1 = tf.nn.sigmoid(tf.matmul(X, W1))

# 텐서플로우에 내장된 함수를 이용하여 dropout 을 적용합니다.
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
L2 = tf.nn.sigmoid(tf.matmul(L1, W2))
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([256, test_label.shape[1]], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L2, W3))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# 학습한 모델을 불러 옵니다.
init = tf.global_variables_initializer()

save_file = './output/train_model.ckpt'


In [5]:


with tf.Session(config=config) as sess:
    saver = tf.train.Saver(tf.global_variables())
    print('모델 불러 옵니다.')
    saver.restore(sess, save_file)
    pred = tf.argmax(model, 1)
    
    is_correct = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    prediction = sess.run(pred, feed_dict={X: org_data, Y: test_label, keep_prob: 1})
    print('정확도 :', sess.run(accuracy, feed_dict={X: org_data, Y: test_label, keep_prob: 1}))

# 학습  종료 합니다.

모델 불러 옵니다.
INFO:tensorflow:Restoring parameters from ./output/train_model.ckpt
정확도 : 0.7236432
